In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
cases = pd.read_csv("WHO-COVID-19-global-data.csv")
vaccines = pd.read_csv("vaccination-data.csv")
cases.head(2)

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-03,AF,Afghanistan,EMRO,0,0,0,0
1,2020-01-04,AF,Afghanistan,EMRO,0,0,0,0


In [3]:
vaccines.head(2)

,COUNTRY,ISO3,WHO_REGION,DATA_SOURCE,DATE_UPDATED,TOTAL_VACCINATIONS,PERSONS_VACCINATED_1PLUS_DOSE,TOTAL_VACCINATIONS_PER100,PERSONS_VACCINATED_1PLUS_DOSE_PER100,PERSONS_FULLY_VACCINATED,PERSONS_FULLY_VACCINATED_PER100,VACCINES_USED,FIRST_VACCINE_DATE,NUMBER_VACCINES_TYPES_USED,PERSONS_BOOSTER_ADD_DOSE,PERSONS_BOOSTER_ADD_DOSE_PER100
0,Afghanistan,AFG,EMRO,REPORTING,2023-04-24,16844011.0,14732514,43.269,37.845,14018650,36.011,"AstraZeneca - Vaxzevria,Beijing CNBG - BBIBP-C...",2021-02-22,11.0,1077102.0,2.767
1,Albania,ALB,EURO,REPORTING,2023-03-19,3070468.0,1347054,106.700,47.332,1276432,44.851,"AstraZeneca - Vaxzevria,Gamaleya - Gam-Covid-V...",2021-01-13,5.0,395384.0,13.893


## Data Preprocessing

In [4]:
vaccines.columns = vaccines.columns.map(str.title)

In [5]:
# Renaming country names to be compatible across files.

replacements = {"Russian Federation": "Russia",
                "Iran (Islamic Republic of)": "Iran",
                "Türkiye": "Turkey",
                "The United Kingdom": "United Kingdom",
                "Republic of Korea": "South Korea",
                "Democratic People's Republic of Korea":"North Korea",
                "Viet Nam": "Vietnam",
                "Bolivia (Plurinational State of)" : "Bolivia",
                "Venezuela (Bolivarian Republic of)":"Venezuela",
                }

cases.replace({"Country": replacements}, inplace=True)
vaccines.replace({"Country": replacements}, inplace=True)

In [6]:
# Shapefile, it contains info about boundaries
shapefile = "custom.geo.json"
# Reading the shapefile
gdf = gpd.read_file(shapefile)
# Subsetting the shape file
gdf = gdf[["admin", "geometry"]].copy()
# Renaming accordingly
gdf.rename(columns={'admin': 'Country'}, inplace=True)
gdf.head(2)

,Country,geometry
0,Costa Rica,"MULTIPOLYGON (((-83.69650 10.93659, -83.68687 ..."
1,Nicaragua,"MULTIPOLYGON (((-85.70174 11.08088, -85.70242 ..."


In [7]:
cases_deaths = cases.groupby("Country").agg({"New_cases": "sum", "New_deaths": "sum"}).reset_index()
cases_deaths.rename(columns={'New_cases': "Total_cases", 'New_deaths': "Total_deaths"}, inplace=True)
cases_deaths.head(3)

,Country,Total_cases,Total_deaths
0,Afghanistan,214112,7888
1,Albania,334090,3604
2,Algeria,271673,6881


In [8]:
# Subsetting vaccine dataset to contain only requried information
vax = vaccines[["Country", "Total_Vaccinations", "Total_Vaccinations_Per100"]].copy()
vax.head(2)

,Country,Total_Vaccinations,Total_Vaccinations_Per100
0,Afghanistan,16844011.0,43.269
1,Albania,3070468.0,106.700


In [9]:
fulldata = cases_deaths.merge(vax, on=["Country"], how="left")
fulldata.shape

(237, 5)

In [10]:
# Merging shape file columns with cases cases_deaths.
df = gdf.merge(fulldata, on="Country", how="left")
df.shape

(248, 6)

## Plotting

In [11]:
import numpy as np
from bokeh.io import show, output_notebook
from bokeh.models import GeoJSONDataSource, HoverTool
from bokeh.plotting import figure
from bokeh.models import Legend

In [12]:
output_notebook()

Loading BokehJS ...

## Total Cases Information

In [13]:
# Define the color range and palette
blue_color_palette = {"0" : "#ebe973",
                      "1 - 5,000" : "#8adae3",
                      "5,001 - 50,000" : "#47c8d6",
                      "50,001 - 500,000": "#169ec7",
                      "500,001 - 5,000,000" : "#158aad", 
                      "5,000,000+" : "#135a70",
                      "Not Applicable" : "#e6edf0"}


df['Color_cases'] = pd.cut(df['Total_cases'], 
                            [-np.inf, 0, 5000, 50000, 500000, 5000000, np.inf], 
                            labels=list(blue_color_palette.values())[:-1]).to_numpy()
df['Color_cases'].fillna(blue_color_palette["Not Applicable"],inplace=True)

In [14]:
geosource = GeoJSONDataSource(geojson=df.to_json())

plot_cases = figure(title="Total Covid Cases by Country",
           width=1400, 
           height=650, 
           tools="wheel_zoom, pan, reset", 
           x_axis_type='mercator', 
           y_axis_type='mercator')

plot_cases.toolbar.logo = None
plot_cases.toolbar_location = "above"
plot_cases.xgrid.grid_line_color = None
plot_cases.ygrid.grid_line_color = None
plot_cases.xaxis.visible = False
plot_cases.yaxis.visible = False
plot_cases.axis.visible = False

patches = plot_cases.patches("xs", "ys", 
                             fill_color="Color_cases", 
                             line_color="black", 
                             line_width=0.5, source=geosource)

hover_cases = HoverTool(tooltips=[
                            ("Country", "@Country"), 
                            ("Total Cases", "@Total_cases"), 
                            ("Total Deaths", "@Total_deaths")])
plot_cases.add_tools(hover_cases)

legend_items = [(label, [plot_cases.square([np.nan], [np.nan], fill_color=color, line_color="black")]) \
                for label, color in blue_color_palette.items()]
legend = Legend(items=legend_items, location="top_right", title="Total Cases")
plot_cases.add_layout(legend, "right")

## Total Deaths Information

In [15]:
# Define the color range and palette

red_color_palette = {"0" : "#e8ccbc",
                     "1 - 50" : "#c26d3c",
                     "51 - 500" : "#ebb494",
                     "501 - 50,000": "#d99066",
                     "50,001 - 50,0000" : "#b35a27", 
                     "50,0000+" : "#873404",
                     "Not Applicable" :"#96928f"}


df['Color_deaths'] = pd.cut(df['Total_deaths'], 
                            [-np.inf, 0, 50, 500, 50000, 500000, np.inf], 
                            labels=list(red_color_palette.values())[:-1]).to_numpy()
df['Color_deaths'].fillna(red_color_palette["Not Applicable"],inplace=True)

In [16]:
geosource = GeoJSONDataSource(geojson=df.to_json())

plot_deaths = figure(title="Total Covid Deaths by Country",
           width=1400, 
           height=650, 
           tools="wheel_zoom, pan, reset", 
           x_axis_type='mercator', 
           y_axis_type='mercator')

plot_deaths.toolbar.logo = None
plot_deaths.toolbar_location = "above"
plot_deaths.xgrid.grid_line_color = None
plot_deaths.ygrid.grid_line_color = None
plot_deaths.xaxis.visible = False
plot_deaths.yaxis.visible = False
plot_deaths.axis.visible = False

patches = plot_deaths.patches("xs", "ys", fill_color="Color_deaths", line_color="black", line_width=0.5, source=geosource)

hover_cases = HoverTool(tooltips=[
                            ("Country", "@Country"), 
                            ("Total Deaths", "@Total_deaths"),
                            ("Total Cases", "@Total_cases")])
plot_deaths.add_tools(hover_cases)

from bokeh.models import Legend


legend_items = [(label, [plot_deaths.square([np.nan], [np.nan], fill_color=color, line_color="black")]) \
                for label, color in red_color_palette.items()]
legend = Legend(items=legend_items, location="top_right", title="Total Deaths")
plot_deaths.add_layout(legend, "right")

### Total Vaccinations Information

In [17]:
green_color_palette = {"0 - 10" : "#c1f7be",
                      "10 - 20" : "#9ef799",
                      "20 - 30" : "#6ed968",
                      "30 - 40" : "#3f963b",
                      "40 - 50" : "#3ad631", 
                      "50 - 60" : "#248f1e",
                      "60 - 70" : "#0f9e08" ,
                      "70 - 80" : "#048508",
                      "80 - 90" : "#045707",
                      "90 - 100" : "#024d05",
                      "Not Applicable" : "#f2faf7"}


df['Color_vax'] = pd.cut(df['Total_Vaccinations_Per100'], 
                         [-np.inf, 10, 20, 30, 40, 50, 60, 70, 80, 90, np.inf], 
                         labels=list(green_color_palette.values())[:-1]).to_numpy()
df['Color_vax'].fillna(green_color_palette["Not Applicable"],inplace=True)

In [18]:
geosource = GeoJSONDataSource(geojson=df.to_json())

plot_vax = figure(title="Total Covid Vaccinations per 100 by Country",
           width=1400, 
           height=650, 
           tools="wheel_zoom, pan, reset", 
           x_axis_type='mercator', 
           y_axis_type='mercator')

plot_vax.toolbar.logo = None
plot_vax.toolbar_location = "above"
plot_vax.xgrid.grid_line_color = None
plot_vax.ygrid.grid_line_color = None
plot_vax.xaxis.visible = False
plot_vax.yaxis.visible = False
plot_vax.axis.visible = False

patches = plot_vax.patches("xs", "ys", fill_color="Color_vax", line_color="black", line_width=0.5, source=geosource)

hover_cases = HoverTool(tooltips=[
                            ("Country", "@Country"), 
                            ("Total Vaccinations per 100", "@Total_Vaccinations_Per100")])
plot_vax.add_tools(hover_cases)

from bokeh.models import Legend


legend_items = [(label, [plot_vax.square([np.nan], [np.nan], fill_color=color, line_color="black")]) \
                for label, color in green_color_palette.items()]
legend = Legend(items=legend_items, location="top_right", title="Total Vaccinations per 100")
plot_vax.add_layout(legend, "right")

In [ ]:
from bokeh.layouts import column
from bokeh.models import CustomJS, RadioButtonGroup

# Define a callback function that toggles the visibility of the plots
toggle_plots = CustomJS(args=dict(plot_cases=plot_cases, plot_deaths=plot_deaths, plot_vax=plot_vax), code="""
    if (cb_obj.active === 0) {
        plot_cases.visible = true;
        plot_deaths.visible = false;
        plot_vax.visible = false;
    } else if (cb_obj.active === 1) {
        plot_cases.visible = false;
        plot_deaths.visible = true;
        plot_vax.visible = false;
    } else if (cb_obj.active === 2) {
        plot_cases.visible = false;
        plot_deaths.visible = false;
        plot_vax.visible = true;
    }
""")

# Create a radio button group with two options
radio_button_group = RadioButtonGroup(labels=["Show Cases", "Show Deaths", "Show Vaccinations"], active=0, css_classes=["radio"])

# Attach the toggle_plots callback to the radio button group
radio_button_group.js_on_change("active", toggle_plots)

# Add the radio button group and the plots to a column layout
layout = column(radio_button_group, plot_cases, plot_deaths, plot_vax)

# Show the layout
show(layout)